---
title: "Auto Detect Score Hand"
author: "Ali Zaidi"
date: "2025-12-1"
categories: [Data Engineering, Automated Annotation, Exploratory Data Analysis]
description: "Watching every swing to find the score is a bit tedious, lets get some functionality to find the frames where the score is provided"
format:
  html:
    code-fold: true
jupyter: python3
---

In [1]:
from fastai.vision.all import *
from eagle_swing.data_class import *
from eagle_swing.animate import *
from eagle_swing.video_utils import *
from eagle_swing.find_landmarks import *
%matplotlib inline

In [2]:
# base_path = '../../../data/full_videos/ymirza/nov16/'#sep14/IMG_1092/keypoint'
# os.listdir(base_path)[:4]

In [3]:
# df = pd.read_csv('../../../data/full_videos/ymirza/ymirza_lbls.csv')
# df.head(1)

In [4]:
# scores = df[df.video_name == 'IMG_1092'].reset_index(drop=True).score.values
# scores

In [5]:
# #pkl_file_path, video_path  = get_files(base_path)
# pkl_file_paths  = get_files(base_path, extensions='.pkl')
# #pkl_file_path, video_path 
# pkl_file_paths

In [6]:
# date_folder = "sep14"
# video_name = "IMG_1086"
# problem_path = f'../../../data/full_videos/ymirza/{date_folder}/{video_name}/keypoints/{video_name}.pkl'
# #problem_path = '../../../data/full_videos/ymirza/sep14/IMG_1091/keypoints/IMG_1091.pkl'
# #kp_extracted = KpExtractor(pkl_file_path)
# kp_extracted = KpExtractor(problem_path)
# kps = kp_extracted.kps
# kps.shape

In [7]:
# kp_extracted = KpExtractor(problem_path)
# #kp_extracted = KpExtractor(pkl_file_paths[0])
# sequence_dict, frame_mask = find_score_hand(kp_extracted, min_consecutive=75)
# frame_mask.sum(), sequence_dict.__len__(), sequence_dict[0]

In [8]:
ls ../../../data/full_videos/ymirza/oct25//

IMG_1183/     IMG_1185.MOV  IMG_1187.mp4  IMG_1190/     IMG_1192.MOV
IMG_1183.MOV  IMG_1185.mp4  IMG_1188/     IMG_1190.MOV  IMG_1192.mp4
IMG_1183.mp4  IMG_1186/     IMG_1188.MOV  IMG_1190.mp4  IMG_1193/
IMG_1184/     IMG_1186.MOV  IMG_1188.mp4  IMG_1191/     IMG_1193.MOV
IMG_1184.MOV  IMG_1186.mp4  IMG_1189/     IMG_1191.MOV  IMG_1193.mp4
IMG_1184.mp4  IMG_1187/     IMG_1189.MOV  IMG_1191.mp4
IMG_1185/     IMG_1187.MOV  IMG_1189.mp4  IMG_1192/


In [500]:
#get_files('../../../data/full_videos/ymirza/oct25', extensions='.pkl')

In [9]:
date_folder = "oct25"
video_name = "IMG_1184"
problem_path = f'../../../data/full_videos/ymirza/{date_folder}/{video_name}/keypoints/{video_name}.pkl'
#problem_df = df[df.video_name == video_name].reset_index(drop=True)
#problem_df.shape

In [10]:
kp_ext = KpExtractor(problem_path)
all_higher_wrists = find_all_higher_wrist_idxs(kp_ext.kps)
first_higher_list = find_each_first_higher_wrist(all_higher_wrists)
#first_higher_list = first_higher_list[:6].append(first_higher_list[-2])
before_increment = 60
after_increment = 900
potential_swing_idxs = [(idx-before_increment, idx+after_increment) for idx in first_higher_list]
len(potential_swing_idxs), potential_swing_idxs[:3]

(11, [(1784, 2744), (2404, 3364), (3998, 4958)])

In [11]:
swings_holder = []
for start_idx, end_idx in potential_swing_idxs:
    frames, fps = get_frames(f"{problem_path[:-3]}mp4",
                          start_idx=start_idx,
                          num_frames=end_idx-start_idx,
                          resize_dim=(512, 512),
                            )
    #print(f"frames per second for this are: {fps}")
    swings_holder.append(frames)

100%|██████████████████| 960/960 [00:02<00:00, 425.49it/s]


In [98]:
# good_idxs = [1, 3, 
            4, 5
#                6, 8, 9]
# good_swings = [x for idx, x in enumerate(swings_holder) if idx in good_idxs]
# final_higher_list = [x for idx, x in enumerate(first_higher_list) if idx in good_idxs]
# print(final_higher_list)

In [114]:
first_higher_list

[983, 3197, 4733, 5334, 7040, 7641, 9339, 11131]

In [13]:
#view_videos_grid(good_swings)
#view_videos_grid(swings_holder[1:7])
#view_videos_grid(swings_holder[3:])

In [112]:
hands_holder = get_hands_from_video(problem_path, min_consecutive=30);

issue at index 0


In [86]:
#view_videos_grid(hands_holder)

In [104]:
correct_scores = [1, 5, 5,
                  2, 5, 1, 4, 1]
correct_idxs = np.arange(len(correct_scores))
video_names = [video_name] * len(correct_scores)
pkl_paths = [f"{date_folder}/{video_name}/keypoints/{video_name}.pkl"] * len(correct_scores)
new_problem_df = pd.DataFrame([video_names, correct_idxs, correct_scores, pkl_paths],
                             index=['video_name', 'swing_idx', 'score', 'pkl_path']).T
clip_names = []
for idx, row in new_problem_df.iterrows():
    clip_name = f"{row.video_name}_swing_{row.swing_idx}_score_{row.score}"
    clip_names.append(clip_name)
new_problem_df['clip_name'] = clip_names
new_problem_df.head(2)

,video_name,swing_idx,score,pkl_path,clip_name
0,IMG_1192,0,1,oct25/IMG_1192/keypoints/IMG_1192.pkl,IMG_1192_swing_0_score_1
1,IMG_1192,1,5,oct25/IMG_1192/keypoints/IMG_1192.pkl,IMG_1192_swing_1_score_5


In [105]:
# keep_idx_list = final_higher_list#first_higher_list[1:]
# new_problem_df['first_higher_wrists_backswing_frame'] = keep_idx_list
# save_csv_path = f"{Path(problem_path).parent}/clean_lbls.csv"
# new_problem_df.to_csv(save_csv_path, index=False)
# save_csv_path

In [106]:
keep_idx_list = first_higher_list
#keep_idx_list = final_higher_list#first_higher_list[1:]
# one_list = first_higher_list[1:6]
# keep_idx_list = one_list.append(first_higher_list[7])

In [107]:
new_problem_df['first_higher_wrists_backswing_frame'] = keep_idx_list
save_csv_path = f"{Path(problem_path).parent}/clean_lbls.csv"
new_problem_df.to_csv(save_csv_path, index=False)
save_csv_path

'../../../data/full_videos/ymirza/oct25/IMG_1192/keypoints/clean_lbls.csv'

We have the start and end frames where one wrist above the shoulder and elbow while the other is below the hip and knee on the opposite side, lets index into the areas where this happens and find the frame where the hand is at it's highest point on the y-axis, from there we can clip X+/- frames around the highest point and only have images of the hands

In [16]:
single_highest_wrist_idx, wrist_xy = find_highest_single_wrist_frame(kp_extracted, sequence_dict, 1)
single_highest_wrist_idx

4404

In [17]:
start_idx, end_idx = get_highest_single_wrist_bounding_idxs(single_highest_wrist_idx, 60)
start_idx, end_idx

(4344, 4434)

In [18]:
one_kps = get_highest_wrist_kps(kp_extracted, start_idx, end_idx)
one_frames, fps = get_highest_wrist_frames(kp_extracted, start_idx, end_idx)
one_frames.shape

100%|████████████| 90/90 [00:00<00:00, 287.13it/s]


(90, 1920, 1080, 3)

In [53]:
nov16_score_dict = {
                    "IMG_1274" : [4, 2, 4, 3, 2, 4, 2, 5, 2, 5, 2, 2, 3, 4],
                    "IMG_1272" : [1, 3, 5, 2, 2, 2, 5, 1],
                    "IMG_1273" : [2, 2, 4, 3, 4, 5, 5, 4, 5, 4, 3, 2, 5],
                    }

In [70]:
df_holder = []
for video_key in nov16_score_dict.keys():
    scores = nov16_score_dict[video_key]
    swing_idxs = np.arange(len(scores))
    video_names = [video_key] * len(scores)
    df_holder.append(pd.DataFrame(zip(video_names, swing_idxs, scores), 
                       columns=['video_name','swing_index','score']), 
          )

In [88]:
nov16_df = pd.concat(df_holder).reset_index(drop=True)
nov16_df['pkl_path'] = nov16_df.video_name.map(lambda x: f"nov16/{x}/keypoints/{x}.pkl")
clip_names = []
for idx, row in nov16_df.iterrows():
    clip_names.append(f"{row.video_name}_swing_{row.swing_index}_score_{row.score}")
nov16_df['clip_name'] = clip_names
nov16_df.shape

(35, 5)

In [55]:
lbl_df.head(2)

,clip_name,video_name,swing_index,score,pkl_path
0,IMG_1015_swing_0_score_3,IMG_1015,0,3,aug9/IMG_1015/keypoints/IMG_1015.pkl
1,IMG_1015_swing_1_score_1,IMG_1015,1,1,aug9/IMG_1015/keypoints/IMG_1015.pkl


In [98]:
! ls ../../../data/full_videos/ymirza/ymirza_lbls.csv

aug9		     jun8		nov16  Untitled1.ipynb
hard_bake_videos.py  label_videos.py	oct25  Untitled.ipynb
index.ipynb	     new_analyze.ipynb	sep14  ymirza_lbls.csv


In [99]:
nov16_df.to_csv('../../../data/full_videos/ymirza/nov16/lbls.csv', index=False)

In [90]:
pd.concat([lbl_df, nov16_df]).reset_index(drop=True)

,clip_name,video_name,swing_index,score,pkl_path
0,IMG_1015_swing_0_score_3,IMG_1015,0,3,aug9/IMG_1015/keypoints/IMG_1015.pkl
1,IMG_1015_swing_1_score_1,IMG_1015,1,1,aug9/IMG_1015/keypoints/IMG_1015.pkl
2,IMG_1015_swing_2_score_3,IMG_1015,2,3,aug9/IMG_1015/keypoints/IMG_1015.pkl
3,IMG_1015_swing_3_score_3,IMG_1015,3,3,aug9/IMG_1015/keypoints/IMG_1015.pkl
4,IMG_1015_swing_4_score_2,IMG_1015,4,2,aug9/IMG_1015/keypoints/IMG_1015.pkl
...,...,...,...,...,...
225,IMG_1273_swing_8_score_5,IMG_1273,8,5,nov16/IMG_1273/keypoints/IMG_1273.pkl
226,IMG_1273_swing_9_score_4,IMG_1273,9,4,nov16/IMG_1273/keypoints/IMG_1273.pkl
227,IMG_1273_swing_10_score_3,IMG_1273,10,3,nov16/IMG_1273/keypoints/IMG_1273.pkl
228,IMG_1273_swing_11_score_2,IMG_1273,11,2,nov16/IMG_1273/keypoints/IMG_1273.pkl


In [40]:
def get_one_hand(kp_extracted,
                 sequence_dict,
                 idx=0, 
                 increment_frame_count_before=30,
                 increment_frame_count_after=60,
                 animate=True,
                 return_hand_only=False):    
    
    single_highest_wrist_idx, wrist_xy = find_highest_single_wrist_frame(kp_extracted, 
                                                                         sequence_dict, 
                                                                         idx)
    start_idx, end_idx = get_highest_single_wrist_bounding_idxs(single_highest_wrist_idx, 
                                                                increment_frame_count_before,
                                                               increment_frame_count_after)
    one_kps = get_highest_wrist_kps(kp_extracted, start_idx, end_idx)
    one_frames, fps = get_highest_wrist_frames(kp_extracted, start_idx, end_idx)
    
    if animate:
        animator = animate_keypoints(one_kps, dark_mode=True)
        display(HTML(animator.to_jshtml()))
        
    if return_hand_only:
        hand_holder = []
        # Ensure we just get the relevant wrist slice
        right_wrist_xy = kp_extracted.r_wrist[start_idx: end_idx, :2]
                
        for i in range(len(right_wrist_xy)):
            x_wrist = int(right_wrist_xy[i][0])
            y_wrist = int(right_wrist_xy[i][1])
            
            # Swap X/Y and handle boundary checks (clamp to 0)
            y1 = max(0, y_wrist - 200)
            y2 = y_wrist + 100
            x1 = max(0, x_wrist - 100)
            x2 = x_wrist + 100
            
            # Index as [Frame, Y (Rows), X (Cols)]
            crop = one_frames[i, y1:y2, x1:x2]
            hand_holder.append(crop)
            
        return np.stack(hand_holder) 
        
    return one_frames

In [41]:
def get_hands_from_video(pkl_file_path,
                        min_consecutive=75,
                        animate=False):
    kp_extracted = KpExtractor(pkl_file_path)
    sequence_dict, frame_mask = find_score_hand(kp_extracted, 
                                                min_consecutive=min_consecutive)
     
    holder = []
    for x in range(len(sequence_dict)):
       try:
           holder.append(get_one_hand(kp_extracted,
                                  sequence_dict,
                                  idx=x,
                                  increment_frame_count_before=0,
                                  increment_frame_count_after=60,
                                  animate=animate, 
                                  return_hand_only=True))
       except:
            print(f'issue at index {x}')
    return holder

In [43]:
hands_holder = get_hands_from_video(pkl_file_paths[0])
view_videos_grid(hands_holder)

100%|████████████| 60/60 [00:00<00:00, 284.00it/s]

In [44]:
hands_holder = get_hands_from_video(pkl_file_paths[1])
view_videos_grid(hands_holder)

100%|████████████| 60/60 [00:00<00:00, 283.84it/s]


In [45]:
hands_holder = get_hands_from_video(pkl_file_paths[2])
view_videos_grid(hands_holder)

100%|████████████| 60/60 [00:00<00:00, 275.04it/s]

Path('../../../data/full_videos/ymirza/nov16/IMG_1274/keypoints/IMG_1274.pkl')

In [216]:
#view_videos(holder[6:])

In [253]:
df.video_name.unique()[10]

'IMG_0860'

In [49]:
parent_path = '../../../data/full_videos/ymirza/'

In [71]:
hand_dataset_parent_dir = '../../../data/score_hands'
hand_dataset_parent_dir

'../../../data/score_hands'

In [65]:
#df.pkl_path.unique().__len__()

In [78]:
df.iloc[1]


clip_name                  IMG_1015_swing_1_score_1
video_name                                 IMG_1015
swing_index                                       1
score                                             1
pkl_path       aug9/IMG_1015/keypoints/IMG_1015.pkl
Name: 1, dtype: object

In [83]:
df[df['video_name'] == df.iloc[0].video_name].score.values

array([3, 1, 3, 3, 2, 2, 1, 4, 1, 2, 4, 4, 2])

In [47]:
lbl_df = pd.read_csv('../../../data/full_videos/ymirza/ymirza_lbls.csv')
lbl_df.head(2)

,clip_name,video_name,swing_index,score,pkl_path
0,IMG_1015_swing_0_score_3,IMG_1015,0,3,aug9/IMG_1015/keypoints/IMG_1015.pkl
1,IMG_1015_swing_1_score_1,IMG_1015,1,1,aug9/IMG_1015/keypoints/IMG_1015.pkl


In [284]:
def save_hand_score_frames(video_name, 
                           df=None,
                           parent_path=None,
                           save_path_parent_dir=None,
                           min_consecutive=75,
                          ):
    if df is None:
        df = pd.read_csv('../../../data/full_videos/ymirza/ymirza_lbls.csv')
#    return df.video_name
    row = df[df.video_name == video_name].reset_index(drop=True).iloc[0]                           
    if parent_path is None:
        parent_path = '../../../data/full_videos/ymirza/'
    if save_path_parent_dir is None:
        save_path_parent_dir = '../../../data/score_hands'
    pkl_path = f'{parent_path}/{row.pkl_path}'
    try:
        holder = get_hands_from_video(pkl_path, min_consecutive)
    except:
        print(f'issue with {row}')
    save_fpath_names = df[df['video_name'] == row.video_name].clip_name.values
    save_paths = [f"{save_path_parent_dir}/{save_fpath}.npy" for save_fpath in save_fpath_names]
    for idx in range(len(save_paths)):
        try:
            np.save(file=save_paths[idx], arr=holder[idx])
        except:
            print(f'having an issue with {save_paths[idx]}')
            print(f'the array has shape{holder[idx].shape}')

In [248]:
df.video_name.unique()[24:]

array(['IMG_1091', 'IMG_1092', 'IMG_1093'], dtype=object)

In [250]:
# for img_name in df.video_name.unique()[24:]:
#     save_hand_score_frames(img_name)

In [252]:
view_videos(np.load('../../../data/score_hands/IMG_1088_swing_2_score_3.npy'))

In [131]:
#pd.read_csv('../../../data/full_videos/ymirza/ymirza_lbls.csv')

In [161]:
df[df.video_name == 'IMG_1018'].pkl_path.iloc[0]

'aug9/IMG_1018/keypoints/IMG_1018.pkl'

In [275]:
df.video_name.unique()[6:19]

array(['IMG_0848', 'IMG_0849', 'IMG_0851', 'IMG_0858', 'IMG_0860',
       'IMG_0852', 'IMG_0853', 'IMG_0854', 'IMG_0855', 'IMG_0856',
       'IMG_0857', 'IMG_0859', 'IMG_0861'], dtype=object)

In [299]:
# problem_video = df.video_name.unique()[18]
# save_hand_score_frames(problem_video, min_consecutive=15)

In [276]:
problem_video = 'IMG_0860'
problem_video = df.video_name.unique()[6]
problem_pkl_path = df[df.video_name == problem_video].pkl_path.iloc[0]
full_path = f'{parent_path}/{problem_pkl_path}'
second_holder = get_hands_from_video(f'{parent_path}/{problem_pkl_path}',
                                    min_consecutive=15)

100%|████████████| 60/60 [00:00<00:00, 288.30it/s]


issue at index 5


100%|████████████| 60/60 [00:00<00:00, 286.73it/s]


In [277]:
scores = df[df.video_name == problem_video].score
scores

64    2
65    5
66    5
67    4
68    4
69    4
70    4
Name: score, dtype: int64

In [282]:
view_videos(second_holder[5:])

In [13]:
def find_highest_single_wrist_frame(KpExtractor_item, sequence_dict, idx):
    start_idx = sequence_dict[idx]['start_frame']
    end_idx = sequence_dict[idx]['end_frame']
    wrist_idx = sequence_dict[idx]['wrist_above']
    if sequence_dict[idx]['wrist_above'] == 'right':
        wrist_xy = KpExtractor_item.r_wrist[start_idx:end_idx][:, :2]
        top_frame_idx = np.argmin(wrist_xy[:, 1])
    else:
        wrist_xy = KpExtractor_item.l_wrist[start_idx:end_idx][:, 2]
        top_frame_idx = np.argmin(wrist_xy[:, 1])        
    video_top_frame_idx = start_idx + top_frame_idx
    return video_top_frame_idx, wrist_xy

def get_highest_single_wrist_bounding_idxs(highest_frame, 
                                           increment_frame_count_before=30,
                                           increment_frame_count_after=30,
                                          ):
    start_idx = highest_frame - increment_frame_count_before
    end_idx = highest_frame + increment_frame_count_after
    return start_idx, end_idx

def get_highest_wrist_kps(KpExtractor_item, start_idx, end_idx):
    return KpExtractor_item.kps[start_idx:end_idx]
    
def get_highest_wrist_frames(KpExtractor_item, start_idx, end_idx):
    video_path = KpExtractor_item.video_path
    frames, fps = get_frames(video_path, 
                        start_idx=start_idx,
                        num_frames=end_idx-start_idx,
                        resize_dim=None)
    return frames, fps

def get_highest_hand_crop(frames, 
                          wrist_xy,
                          increment=50):
    handup_frames = []
    for idx in range(len(wrist_xy)):
        wrist_x = int(wrist_xy[idx, 0])
        wrist_y = int(wrist_xy[idx, 1])
        print(wrist_y<frames.shape[2])
        this_frame = frames[idx, 
                             wrist_x - increment : wrist_x + increment,
                             wrist_y - increment : wrist_y + increment
                             ]
#        print(this_frame.shape)
        if idx == 10:
            break
        handup_frames.append(this_frame)
    return handup_frames

In [14]:
def get_handcrops(pkl_file_path, 
                 min_consecutive_high_frames=75,
                 crop_bounds=60,
                ):
    kp_extracted = KpExtractor(pkl_file_path)
    sequence_dict, frame_mask = find_score_hand(kp_extracted, 
                                                min_consecutive=min_consecutive_high_frames)
    hand_crop_holder = []
    for seq_idx in range(len(sequence_dict)):
        single_highest_wrist_idx, wrist_xy = find_highest_single_wrist_frame(kp_extracted, 
                                                                             sequence_dict, 
                                                                             seq_idx)
        start_idx, end_idx = get_highest_single_wrist_bounding_idxs(single_highest_wrist_idx, 
                                                                    crop_bounds)
        frames, fps = get_frames(kp_extracted.video_path, 
                                 num_frames=end_idx-start_idx, 
                                 start_idx=start_idx, 
                                 resize_dim=None)
        wrist_xy = kp_extracted.r_wrist[start_idx:end_idx]
        hand_crop = get_highest_hand_crop(frames=frames,
                                          wrist_xy=wrist_xy,
                                          increment=100)
        hand_crop_holder.append(hand_crop)
    return hand_crop_holder